In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import sys
import copy
import re
import os
from pathlib import Path
from itertools import chain
from functools import partial

import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import DateOffset, BDay
from pandas.tseries.frequencies import to_offset
# from clarion import positions

from api.gui.base import View, Tabs, Store
from api.utils import parse_offset
from api.gui.market_monitor_h import CurvesViewer, Pivot, get_issuers_curves, get_bond_ref, get_bday, get_history_table
from api.gui.max_diff_h import LowestFieldCurve
from api.gui.visual_h import Visual
from api.gui.fwd_pair_h import Forward
from api.gui.fwd_pivot_h import PivotForward
from api.blpw import BlpQuery
from api.bbg import BBGField, BBGOverrides
from api.data.base import data_path, tables
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import (
    apply_function, 
    add_years_to_maturity, 
    add_maturity_bucket,
    add_custom_rating,
    add_rating_bucket,
    add_geo_area
)


#### Bloomberg Connection

In [ ]:
bq = BlpQuery(timeout=50000).start()

#### Settings

In [ ]:
index_name = 'CDX EM CDSI S37 5Y PRC Corp'
pcs = 'MSG1'
sep = ' '

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [ ]:
new_cdx_mapping = {
    "PEOPLE'S REPUBLIC OF CHINA": "CCHIN1U5 Curncy",
    "UNITED MEXICAN STATES": "CMEX1U5 Curncy",
    "FEDERATIVE REPUBLIC OF BRAZIL": "CBRZ1U5 Curncy",
    "REPUBLIC OF TURKEY": "CTURK1U5 Curncy",
    "REPUBLIC OF SOUTH AFRICA": "CSOAF1U5 Curncy",
    "REPUBLIC OF INDONESIA": "CINO1U5 Curncy",
    "REPUBLIC OF COLOMBIA": "CCOL1U5 Curncy",
    "REPUBLIC OF CHILE": "CCHIL1U5 Curncy",
    "REPUBLIC OF THE PHILIPPINES": "CPHIL1U5 Curncy",
    "MALAYSIA": "CMLAY1U5 Curncy",
    "KINGDOM OF SAUDI ARABIA": "CT965307 Curncy",
    "REPUBLIC OF PERU": "CPERU1U5 Curncy",
    "STATE OF QATAR": "CQTA1U5 Curncy",
    "ARGENTINE REPUBLIC": "CT350188 Curncy",
    "REPUBLIC OF PANAMA": "CPAN1U5 Curncy",
    "ARAB REPUBLIC OF EGYPT": "CEGY1U5 Curncy",
    "ABU DHABI": "CX855707 Curncy",
    "SULTANATE OF OMAN": "CT991547 Curncy",
}

#### Index Members

In [ ]:
def get_index_members(index):
    
    try:
        bq
    except NameError:
        bq = BlpQuery(timeout=50000).start()
        
    members = bq.bds(
        security=index,
        field='indx_members'
    ).iloc[:, :-1].set_axis(['issuer', 'wgt', 'currency', 'seniority', 'security'], axis=1)
    members['security'] = members['security'] + ' Curncy'
    members['wgt'] = members['wgt'] / 100
    return members

In [ ]:
def parse_new_cdx_members(mapping=None):
    mapping = mapping or new_cdx_mapping
    
    wgt = pd.read_clipboard()

    for i in [1, 2]:
        wgt.iloc[:, i] = wgt.iloc[:, i].str.replace('%', '').astype(float) / 100
    
    wgt = wgt.set_axis(['security', '', 'wgt'], axis=1)
    wgt['security'] = wgt['security'].replace(mapping)
    return wgt[['security', 'wgt']]

In [ ]:
def get_members_data(members, pcs=None, sep=None):
        pcs = pcs or 'MSG1'
        sep = sep or ' '
        fields = [
            'security_des',
            'issuer_bulk',
            'crncy',
            'tenor',
            'upfront_format_indicator',
            'sw_spread',
            'px_last',
            'upfront_last',
            # '5y_mid_cds_spread',
        ]
        cdx_data = bq.bdp(
            securities=[*members['security']],
            fields=fields,
            pcs=pcs,
            sep=sep
        )
        
        return cdx_data.rename({'px_last': 'spread', 'upfront_format_indicator': 'is_upfront', 'upfront_last': 'upfront'}, axis=1)

In [ ]:
s1 = 37
s2 = 38

In [ ]:
def on_change_change_label(widget, event, payload):
    if widget.label == 's1':
        widget.label = 's2'
    elif widget.label == 's2':
        widget.label = 's1'


def on_click_get_members_and_data(widget, event, payload, self):
    widget.loading = True
    try:
        self.get_all_members_and_data()
        display(self.dg_all)
    except Exception as e:
        widget.color = 'error'
        raise e
    else:
        widget.color = 'success'
    finally:
        widget.loading = False
        
    
def on_click_get_intrinsic(widget, event, payload, self):
    widget.loading = True
    try:
        self.get_intrinsic()
        display(self.get_html())
    except Exception as e:
        widget.color = 'error'
        raise e
    else:
        widget.color = 'success'
    finally:
        widget.loading = False        
        

def copy_sel(dg):
    sels = [dg.data.iloc[sel['r1']: sel['r2'] + 1, sel['c1']: sel['c2'] + 1] for sel in dg.selections]
    try:
        cols = np.vstack([sel.columns for sel in sels])
        test = np.all(cols == cols[0])
        if test:
             res = pd.concat(sels)
    except Exception as e:
        print(e)

        try:
            res = pd.concat(sels, axis=1)
        except Exception as e:
            print(e)

    res.to_clipboard()
    return res       

In [ ]:
class CDXComp(View):
    def get_index_name(self, s):
        return f'CDX EM CDSI S{s} 5Y PRC Corp'
    
    def get_html(self):
        return w.HTML(f"<b> S{self.s1}</b>: {i1:.2f} <br> <b> S{self.s2}</b>: {i2:.2f}")
    
    # def __init__(self, s1, s2, quote='s1'):
    #     if s1 > s2:
    #         s1, s2 = s2, s1
    #     self.s1 = s1
    #     self.s2 = s2
    #     self.quote = quote
    
    def make_widgets(self, **kwargs):
        s1_tf = v.TextField(
            v_model=None,
            label='s1',
            clearable=True,
            dense=True,
            type="number",
            outlined=True
        )

        s2_tf = v.TextField(
            v_model=None,
            label='s2',
            clearable=True,
            dense=True,
            type="number",
            outlined=True
        )


        s_sw = v.Switch(
            v_model=False,
            label='s1',
            dense=True,
            # class_='d-inline',
            hide_details=True
        )

        dl_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-2',
            children=[v.Icon(children=['mdi-download'])]
        )


        dl_btn2 = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-2',
            children=[v.Icon(children=['mdi-download'])]
        )
        
        self.s1_tf = s1_tf
        self.s2_tf = s2_tf
        self.s_sw = s_sw
        self.dl_btn = dl_btn
        self.dl_btn2 = dl_btn2
        self.out = w.Output()
        self.out2 = w.Output()
        
    def make_view(self, **kwargs):
        
        s1_tf = self.s1_tf
        s2_tf = self.s2_tf
        s_sw = self.s_sw
        dl_btn = self.dl_btn
        dl_btn2 = self.dl_btn2
        out = self.out
        out2= self.out2
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=1.0,
                            children=[s_sw],
                            class_="my-0 py-0",
                            # align_self='center'
                        ),
                        v.Col(
                            cols=1.0,
                            children=[s1_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=1.0,
                            children=[s2_tf],
                            class_="my-0 py-0"
                        ),

                        v.Col(
                            cols=3,
                            children=[dl_btn, dl_btn2],
                            class_="my-0 py-0",
                            # align_self='center'
                        ),

                    ],
                    # align='center',
                    # justify='center'
                ),
            ],
            # fill_height=True,
            # fluid=True
        )
        
        view = w.VBox(
            children=[
                param_box,
                out2,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view
    
    def link(self, **kwargs):
        s_sw = self.s_sw
        dl_btn = self.dl_btn
        dl_btn2 = self.dl_btn2
        out = self.out
        out2 = self.out2
        
        s_sw.on_event('change', on_change_change_label)
        
        dl_btn.on_event(
            'click',
            partial(
                out.capture(clear_output=True)(on_click_get_members_and_data),
                self=self
            )
        )
        
        dl_btn2.on_event(
            'click',
            partial(
                out2.capture(clear_output=True)(on_click_get_intrinsic),
                self=self
            )
        )
        
        super().link()
        
    @property
    def s1(self):
        s1 = int(self.s1_tf.v_model)
        s2 = int(self.s2_tf.v_model)
        if s1 > s2:
            s1, s2 = s2, s1
        return s1
    
    @property
    def s2(self):
        s1 = int(self.s1_tf.v_model)
        s2 = int(self.s2_tf.v_model)
        if s1 > s2:
            s1, s2 = s2, s1
        return s2
    
    @property
    def quote(self):
        return 's1' if not self.s_sw.v_model else 's2'
    
    def get_all_members_and_data(self, s1=None, s2=None):
        s1 = s1 or self.s1
        s2 = s2 or self.s2
        
        if s1 > s2:
            s1, s2 = s2, s1
        index_name1 = f'CDX EM CDSI S{s1} 5Y PRC Corp'
        index_name2 = f'CDX EM CDSI S{s2} 5Y PRC Corp'
        old_members = get_index_members(index_name1)
        
        try:
            new_members = get_index_members(index_name2)
        except Exception:
            new_members = parse_new_cdx_members()
        
        old_data = get_members_data(old_members) 
        new_data = get_members_data(new_members)
        
        all_data = pd.concat([old_data, new_data.loc[~new_data['security'].isin(old_data['security']), :]]).reset_index(drop=True)
        
        self.s1_members = old_members
        self.s2_members = new_members
        self.s1_data = old_data
        self.s2_data = new_data
        self.all_data = all_data
        
        self.dg_all = self.make_dg1(all_data)
        
    def get_intrinsic(self):
        cdx_data = self.dg_all.data.copy()
        s1_data = cdx_data.loc[cdx_data['security'].isin(self.s1_members['security']), :]
        s2_data = cdx_data.loc[cdx_data['security'].isin(self.s2_members['security']), :]

        if self.quote == 's1':
            s2_data.loc[s2_data['is_upfront'] == 0, 'spread'] = (s2_data['spread'] + s2_data['roll'].fillna(0).astype(float)).loc[s2_data['is_upfront'] == 0]
            s2_data.loc[s2_data['is_upfront'] == 1, 'upfront'] = (s2_data['upfront'] + s2_data['roll'].fillna(0).astype(float)).loc[s2_data['is_upfront'] == 1]
        else:
            s1_data.loc[s1_data['is_upfront'] == 0, 'spread'] = (s1_data['spread'] - s1_data['roll'].fillna(0).astype(float)).loc[s1_data['is_upfront'] == 0]
            s1_data.loc[s1_data['is_upfront'] == 1, 'upfront'] = (s1_data['upfront'] - s1_data['roll'].fillna(0).astype(float)).loc[s1_data['is_upfront'] == 1]

        s1_maturity = bq.bdp(
            securities=[self.get_index_name(s=self.s1)],
            fields=['maturity']
        )['maturity'].squeeze()

        s2_maturity = s1_maturity + pd.tseries.offsets.DateOffset(months=6)

        s1_intrinsic = get_cdx_intrinsic_data(members=self.s1_members, cdx_data=s1_data, maturity=f"{s1_maturity:{bbg_dt_fmt}}")
        s2_intrinsic = get_cdx_intrinsic_data(members=self.s2_members, cdx_data=s2_data, maturity=f"{s2_maturity:{bbg_dt_fmt}}")

        self.s1_intrinsic = s1_intrinsic
        self.s2_intrinsic = s2_intrinsic
        self.s1_dgi = self.make_dg1(s1_intrinsic)
        self.s2_dgi = self.make_dg1(s2_intrinsic)
        
    def make_dg1(self, data):
        if 'accrued' in data:
            data = data.drop('accrued', axis=1)
        renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.2f')"),
            #horizontal_alignment="center",
        )
        
        pct_renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.2%')"),
            #horizontal_alignment="center",
        )

        dg = DataGrid(
            data.assign(roll=None),
            base_column_size=90,
            column_widths={
                'security': 130,
                'security_des': 250,
                'issuer_bulk': 250,
                'cds_quoted_price': 110,
                'cds_quoted_price_net': 110,
            },
            renderers={
                'spread': renderer,
                'upfront': renderer,
                'roll': renderer,
                'cds_quoted_price': renderer,
                'cds_quoted_price_net': renderer,
                'wgt': pct_renderer,
            },
            selection_mode='cell',
            editable=True
        )
        
        return dg

In [653]:
self = CDXComp()
self.s1_tf.v_model = 37
self.s2_tf.v_model = 38
self

In [260]:
class CDXComp:
    def get_index_name(self, s):
        return f'CDX EM CDSI S{s} 5Y PRC Corp'
    
    def get_html(self):
        return w.HTML(f"<b> S{self.s1}</b>: {i1:.2f} <br> <b> S{self.s2}</b>: {i2:.2f}")
    
    def __init__(self, s1, s2, quote='s1'):
        if s1 > s2:
            s1, s2 = s2, s1
        self.s1 = s1
        self.s2 = s2
        self.quote = quote
    
    def get_all_members_and_data(self, s1=None, s2=None):
        s1 = s1 or self.s1
        s2 = s2 or self.s2
        
        if s1 > s2:
            s1, s2 = s2, s1
        index_name1 = f'CDX EM CDSI S{s1} 5Y PRC Corp'
        index_name2 = f'CDX EM CDSI S{s2} 5Y PRC Corp'
        old_members = get_index_members(index_name1)
        
        try:
            new_members = get_index_members(index_name2)
        except Exception:
            new_members = parse_new_cdx_members()
        
        old_data = get_members_data(old_members) 
        new_data = get_members_data(new_members)
        
        all_data = pd.concat([old_data, new_data.loc[~new_data['security'].isin(old_data['security']), :]]).reset_index(drop=True)
        
        self.s1 = s1
        self.s2 = s2
        self.s1_members = old_members
        self.s2_members = new_members
        self.s1_data = old_data
        self.s2_data = new_data
        self.all_data = all_data
        
        self.dg_all = self.make_dg1(all_data)
        
    def get_intrinsic(self):
        cdx_data = self.dg_all.data.copy()
        s1_data = cdx_data.loc[cdx_data['security'].isin(self.s1_members['security']), :]
        s2_data = cdx_data.loc[cdx_data['security'].isin(self.s2_members['security']), :]

        if self.quote == 's1':
            s2_data.loc[s2_data['is_upfront'] == 0, 'spread'] = (s2_data['spread'] + s2_data['roll'].fillna(0).astype(float)).loc[s2_data['is_upfront'] == 0]
            s2_data.loc[s2_data['is_upfront'] == 1, 'upfront'] = (s2_data['upfront'] + s2_data['roll'].fillna(0).astype(float)).loc[s2_data['is_upfront'] == 1]
        else:
            s1_data.loc[s1_data['is_upfront'] == 0, 'spread'] = (s1_data['spread'] - s1_data['roll'].fillna(0).astype(float)).loc[s1_data['is_upfront'] == 0]
            s1_data.loc[s1_data['is_upfront'] == 1, 'upfront'] = (s1_data['upfront'] - s1_data['roll'].fillna(0).astype(float)).loc[s1_data['is_upfront'] == 1]

        s1_maturity = bq.bdp(
            securities=[self.get_index_name(s=self.s1)],
            fields=['maturity']
        )['maturity'].squeeze()

        s2_maturity = s1_maturity + pd.tseries.offsets.DateOffset(months=6)

        s1_intrinsic = get_cdx_intrinsic_data(members=self.s1_members, cdx_data=s1_data, maturity=f"{s1_maturity:{bbg_dt_fmt}}")
        s2_intrinsic = get_cdx_intrinsic_data(members=self.s2_members, cdx_data=s2_data, maturity=f"{s2_maturity:{bbg_dt_fmt}}")

        self.s1_intrinsic = s1_intrinsic
        self.s2_intrinsic = s2_intrinsic
        self.s1_dgi = self.make_dg1(s1_intrinsic)
        self.s2_dgi = self.make_dg1(s2_intrinsic)
        
    def make_dg1(self, data):
        if 'accrued' in data:
            data = data.drop('accrued', axis=1)
        renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.2f')"),
            #horizontal_alignment="center",
        )
        
        pct_renderer = TextRenderer(
            text_value=Expr("format(cell.value, '.2%')"),
            #horizontal_alignment="center",
        )

        dg = DataGrid(
            data.assign(roll=None),
            base_column_size=90,
            column_widths={
                'security': 130,
                'security_des': 250,
                'issuer_bulk': 250,
                'cds_quoted_price': 110,
                'cds_quoted_price_net': 110,
            },
            renderers={
                'spread': renderer,
                'upfront': renderer,
                'roll': renderer,
                'cds_quoted_price': renderer,
                'cds_quoted_price_net': renderer,
                'wgt': pct_renderer,
            },
            selection_mode='cell',
            editable=True
        )
        
        return dg

In [650]:
self.out2

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "HTML(value='<b> S37</b>: 93.25 <br> <b…

In [ ]:
dg.set_cell_value()

In [621]:
dg = self.dg_all

In [627]:
pd.concat([dg.data.iloc[sel['r1']: sel['r2'] + 1, sel['c1']: sel['c2'] + 1] for sel in dg.selections], axis=1)

security              security_des         security  \
key                                                               
1     CEGY1U5 Curncy   EGYPT CDS USD SR 5Y D14              NaN   
2    CT350188 Curncy  ARGENT CDS USD SR 5Y D14              NaN   
3     CBRZ1U5 Curncy  BRAZIL CDS USD SR 5Y D14              NaN   
7                NaN                       NaN  CCHIL1U5 Curncy   
8                NaN                       NaN   CCOL1U5 Curncy   
9                NaN                       NaN   CINO1U5 Curncy   

                security_des  
key                           
1                        NaN  
2                        NaN  
3                        NaN  
7    CHILE CDS USD SR 5Y D14  
8    COLOM CDS USD SR 5Y D14  
9    INDON CDS USD SR 5Y D14

In [625]:
dg.data.iloc[sel['r1']: sel['r2'] + 1, sel['c1']: sel['c2'] + 1]

security              security_des
key                                           
1     CEGY1U5 Curncy   EGYPT CDS USD SR 5Y D14
2    CT350188 Curncy  ARGENT CDS USD SR 5Y D14
3     CBRZ1U5 Curncy  BRAZIL CDS USD SR 5Y D14

In [624]:
sel

{'r1': 1, 'r2': 3, 'c1': 0, 'c2': 1}

In [261]:
self = CDXComp(37, 38)

In [254]:
self.get_all_members_and_data()
self.dg_all

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_size=90, column_widths={…

HTML(value='<b> S37</b>: 93.25 <br> <b> S38</b>: 92.81')

In [258]:
self.s1_dgi

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_size=90, column_widths={…

In [259]:
self.s2_dgi

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_size=90, column_widths={…

In [174]:
s1_df =  pd.read_clipboard(header=None)

In [162]:
roll = pd.read_clipboard(header=None)

In [175]:
s1_df['mid'] = (s1_df[1] + s1_df[2]) / 2
roll['mid'] = (roll[1] + roll[2]) / 2

In [181]:
to_change =    {
    'ABUDHAB': {'value': 0, 'roll': 0},
    'ARGENT': {'value': 69, 'roll': 0.505},
    'BRAZIL': {'value': 246, 'roll': 22.250},
    'CHILE': {'value': 135, 'roll': 0},
    'CHINAGOV': {'value': 0, 'roll': 0},
    'COLOM': {'value': 256.5, 'roll': 23.750},
    'EGYPT': {'value': 0, 'roll': 0},
    'INDON': {'value': 0, 'roll': 0},
    'KSA': {'value': 0, 'roll': 0},
    'MALAYS': {'value': 0, 'roll': 0},
    'MEX': {'value': 156, 'roll': 14.250},
    'OMAN': {'value': 0, 'roll': 0},
    'PANAMA': {'value': 138, 'roll': 13},
    'PERU': {'value': 123, 'roll': 11},
    'PHILIP': {'value': 0, 'roll': 0},
    'QATAR': {'value': 0, 'roll': 0},
    'REPSOU': {'value': 0, 'roll': 0},
    'TURKEY': {'value': 0, 'roll': 0},
    'UKRAIN': {'value': 0, 'roll': 0}
}

In [153]:
self.dg_all.data['security_des'].to_clipboard()

In [ ]:
{
    
}

In [219]:
cdx_data = self.dg_all.data.copy()
s1_data = cdx_data.loc[cdx_data['security'].isin(self.s1_members['security']), :]
s2_data = cdx_data.loc[cdx_data['security'].isin(self.s2_members['security']), :]

if self.quote == 's1':
    s2_data.loc[s2_data['is_upfront'] == 0, 'spread'] = (s2_data['spread'] + s2_data['roll'].fillna(0).astype(float)).loc[s2_data['is_upfront'] == 0]
    s2_data.loc[s2_data['is_upfront'] == 1, 'upfront'] = (s2_data['upfront'] + s2_data['roll'].fillna(0).astype(float)).loc[s2_data['is_upfront'] == 1]
else:
    s1_data.loc[s1_data['is_upfront'] == 0, 'spread'] = (s1_data['spread'] - s1_data['roll'].fillna(0).astype(float)).loc[s1_data['is_upfront'] == 0]
    s1_data.loc[s1_data['is_upfront'] == 1, 'upfront'] = (s1_data['upfront'] - s1_data['roll'].fillna(0).astype(float)).loc[s1_data['is_upfront'] == 1]

s1_maturity = bq.bdp(
    securities=[self.get_index_name(s=self.s1)],
    fields=['maturity']
)['maturity'].squeeze()

s2_maturity = s1_maturity + pd.tseries.offsets.DateOffset(months=6)

s1_intrinsic = get_cdx_intrinsic_data(members=self.s1_members, cdx_data=s1_data, maturity=f"{s1_maturity:{bbg_dt_fmt}}")
s2_intrinsic = get_cdx_intrinsic_data(members=self.s2_members, cdx_data=s2_data, maturity=f"{s2_maturity:{bbg_dt_fmt}}")

self.s1_intrinsic = s1_intrinsic
self.s2_intrinsic = s2_intrinsic

self.s1_dgi = self.make_dg1(s1_intrinsic)
self.s2_dgi = self.make_dg1(s2_intrinsic)

In [223]:
s1_intrinsic = get_cdx_intrinsic_data(members=self.s1_members, cdx_data=s1_data, maturity=f"{s1_maturity:{bbg_dt_fmt}}")
s2_intrinsic = get_cdx_intrinsic_data(members=self.s2_members, cdx_data=s2_data, maturity=f"{s2_maturity:{bbg_dt_fmt}}")

In [205]:
s1_intrinsic

security                security_des  \
0   CX855707 Curncy   ABUDHAB CDS USD SR 5Y D14   
1    CEGY1U5 Curncy     EGYPT CDS USD SR 5Y D14   
2   CT350188 Curncy    ARGENT CDS USD SR 5Y D14   
3    CBRZ1U5 Curncy    BRAZIL CDS USD SR 5Y D14   
4   CT965307 Curncy       KSA CDS USD SR 5Y D14   
5   CMLAY1U5 Curncy    MALAYS CDS USD SR 5Y D14   
6   CCHIN1U5 Curncy  CHINAGOV CDS USD SR 5Y D14   
7   CCHIL1U5 Curncy     CHILE CDS USD SR 5Y D14   
8    CCOL1U5 Curncy     COLOM CDS USD SR 5Y D14   
9    CINO1U5 Curncy     INDON CDS USD SR 5Y D14   
10   CPAN1U5 Curncy    PANAMA CDS USD SR 5Y D14   
11  CPERU1U5 Curncy      PERU CDS USD SR 5Y D14   
12  CSOAF1U5 Curncy    REPSOU CDS USD SR 5Y D14   
13  CPHIL1U5 Curncy    PHILIP CDS USD SR 5Y D14   
14  CTURK1U5 Curncy    TURKEY CDS USD SR 5Y D14   
15   CQTA1U5 Curncy     QATAR CDS USD SR 5Y D14   
16   CUKR1U5 Curncy    UKRAIN CDS USD SR 5Y D14   
17   CMEX1U5 Curncy       MEX CDS USD SR 5Y D14   

                                 issuer_bulk crncy  tenor  is_upfront  \
0   Emirate of Abu Dhabi United Arab Emirate   USD    5.0           0   
1                     Arab Republic of Egypt   USD    5.0           1   
2                     Argentine Republic/The   USD    5.0           1   
3              Federative Republic of Brazil   USD    5.0           0   
4                    Kingdom of Saudi Arabia   USD    5.0           0   
5                     Federation of Malaysia   USD    5.0           0   
6                 People's Republic of China   USD    5.0           0   
7                          Republic of Chile   USD    5.0           0   
8                       Republic of Colombia   USD    5.0           0   
9                      Republic of Indonesia   USD    5.0           0   
10                        Republic of Panama   USD    5.0           0   
11                          Republic of Peru   USD    5.0           0   
12                  Republic of South Africa   USD    5.0           0   
13               Republic of the Philippines   USD    5.0           0   
14                        Republic of Turkey   USD    5.0           0   
15                            State of Qatar   USD    5.0           0   
16                                   Ukraine   USD    5.0           0   
17                     United Mexican States   USD    5.0           0   

    sw_spread     spread       upfront   wgt  cds_quoted_price   accrued  \
0       100.0    46.5000 -2.257847e+00  0.02        102.324057  -24444.0   
1       100.0   948.4475  2.825000e+01  0.02         71.750000  -24444.0   
2       500.0  5220.1650  6.925000e+01  0.02         30.750000 -122222.0   
3       100.0   248.0000  6.122106e+00  0.12         93.955860  -24444.0   
4       100.0    47.5000 -2.213741e+00  0.02        102.279910  -24444.0   
5       100.0    78.5000 -8.545615e-01  0.03        100.919184  -24444.0   
6       100.0    75.2500 -1.005464e+00  0.12        101.059457  -24444.0   
7       100.0   137.0000  1.646535e+00  0.03         98.436944  -24444.0   
8       100.0   258.5000  6.549211e+00  0.06         93.547604  -24444.0   
9       100.0   111.0000  5.484662e-01  0.09         99.535557  -24444.0   
10      100.0   139.5000  1.771560e+00  0.02         98.332612  -24444.0   
11      100.0   124.0000  1.102079e+00  0.03         98.982071  -24444.0   
12      100.0   276.0000  7.203626e+00  0.12         92.873009  -24444.0   
13      100.0    98.5000  1.421085e-14  0.02        100.063638  -24444.0   
14      100.0   746.0000  2.295415e+01  0.12         77.218009  -24444.0   
15      100.0    47.5000 -2.213741e+00  0.02        102.279910  -24444.0   
16      100.0   100.0000  0.000000e+00  0.02        100.000000  -24444.0   
17      100.0   157.0000  2.475693e+00  0.12         97.606764  -24444.0   

    cds_quoted_price_net  
0             102.568497  
1              71.994440  
2              31.972220  
3              94.200300  
4             102.524350  
5             101.163624  
6             101.3

In [194]:
cdx_data

security                security_des  \
key                                                
0    CX855707 Curncy   ABUDHAB CDS USD SR 5Y D14   
1     CEGY1U5 Curncy     EGYPT CDS USD SR 5Y D14   
2    CT350188 Curncy    ARGENT CDS USD SR 5Y D14   
3     CBRZ1U5 Curncy    BRAZIL CDS USD SR 5Y D14   
4    CT965307 Curncy       KSA CDS USD SR 5Y D14   
5    CMLAY1U5 Curncy    MALAYS CDS USD SR 5Y D14   
6    CCHIN1U5 Curncy  CHINAGOV CDS USD SR 5Y D14   
7    CCHIL1U5 Curncy     CHILE CDS USD SR 5Y D14   
8     CCOL1U5 Curncy     COLOM CDS USD SR 5Y D14   
9     CINO1U5 Curncy     INDON CDS USD SR 5Y D14   
10    CPAN1U5 Curncy    PANAMA CDS USD SR 5Y D14   
11   CPERU1U5 Curncy      PERU CDS USD SR 5Y D14   
12   CSOAF1U5 Curncy    REPSOU CDS USD SR 5Y D14   
13   CPHIL1U5 Curncy    PHILIP CDS USD SR 5Y D14   
14   CTURK1U5 Curncy    TURKEY CDS USD SR 5Y D14   
15    CQTA1U5 Curncy     QATAR CDS USD SR 5Y D14   
16    CUKR1U5 Curncy    UKRAIN CDS USD SR 5Y D14   
17    CMEX1U5 Curncy       MEX CDS USD SR 5Y D14   
18   CT991547 Curncy      OMAN CDS USD SR 5Y D14   

                                  issuer_bulk crncy  tenor  is_upfront  \
key                                                                      
0    Emirate of Abu Dhabi United Arab Emirate   USD    5.0           0   
1                      Arab Republic of Egypt   USD    5.0           1   
2                      Argentine Republic/The   USD    5.0           1   
3               Federative Republic of Brazil   USD    5.0           0   
4                     Kingdom of Saudi Arabia   USD    5.0           0   
5                      Federation of Malaysia   USD    5.0           0   
6                  People's Republic of China   USD    5.0           0   
7                           Republic of Chile   USD    5.0           0   
8                        Republic of Colombia   USD    5.0           0   
9                       Republic of Indonesia   USD    5.0           0   
10                         Republic of Panama   USD    5.0           0   
11                           Republic of Peru   USD    5.0           0   
12                   Republic of South Africa   USD    5.0           0   
13                Republic of the Philippines   USD    5.0           0   
14                         Republic of Turkey   USD    5.0           0   
15                             State of Qatar   USD    5.0           0   
16                                    Ukraine   USD    5.0           0   
17                      United Mexican States   USD    5.0           0   
18                          Sultanate of Oman   USD    5.0           0   

     sw_spread    spread       upfront  roll  
key                                           
0        100.0    66.500 -2.257847e+00    20  
1        100.0   978.452  2.900000e+01  None  
2        500.0  5220.165  8.925000e+01    20  
3        100.0   248.000  6.122106e+00  None  
4        100.0    47.500 -2.213741e+00  None  
5        100.0    78.500 -8.545615e-01  None  
6        100.0    75.250 -1.005464e+00  None  
7        100.0   136.000  1.604808e+00  None  
8        100.0   260.000  6.626557e+00  None  
9        100.0   111.000  5.484662e-01  None  
10       100.0   138.000  1.688236e+00  None  
11       100.0   126.000  1.186126e+00  None  
12       100.0   276.000  7.203626e+00  None  
13       100.0    98.500  1.421085e-14  None  
14       100.0   746.000  2.295415e+01  None  
15       100.0    47.500 -2.213741e+00  None  
16       100.0   100.000  0.000000e+00  None  
17       100.0   157.000  2.455088e+00  None  
18       100.0   245.000  6.896518e+00  None

In [138]:
from api.bbg import BBGRequestParams, BBGField, BBGSecurity, BBGOverrides
def get_cdx_intrinsic_data(members, cdx_data=None, pcs=None, sep=None, maturity=None, override_spread=False):
    cdx_data = get_members_data(members, pcs=pcs, sep=sep) if cdx_data is not None else cdx_data
    
    # if 'spread' in members and override_spread:
    #     spread_override = members.loc[members['spread'].isna(), :]
    #     cdx_data.loc[cdx_data['security'].isin([*spread_override['security']]), 'spread'] = [*spread_override['spread']]
    
    cdx_data = cdx_data.merge(members[['security', 'wgt']], on='security', how='left')
    bbg_securities = [
        BBGSecurity(
            id=row['security'], 
            overrides=[
                BBGOverrides(
                    CDS_FLAT_SPREAD=row['spread'],
                    MATURITY=maturity
                ) if maturity else BBGOverrides(CDS_FLAT_SPREAD=row['spread'])
            ]
        ) for index, row in cdx_data.iterrows()]
    bbg_fields = [
        BBGField("cds_quoted_price"),
        BBGField("SW_PAY_ACC_INT",),
    ]
    
    cdx_quoted_price = bq.bdpx(BBGRequestParams(securities=bbg_securities, fields=bbg_fields)).rename({"SW_PAY_ACC_INT": "accrued"}, axis=1)
    cdx_final = cdx_data.merge(cdx_quoted_price[['security', 'cds_quoted_price', "accrued"]], on='security', how='left')
    cdx_final.loc[cdx_final['is_upfront'] == 1, 'cds_quoted_price'] = 100 - cdx_final.loc[cdx_final['is_upfront'] == 1, 'upfront']
    cdx_final["cds_quoted_price_net"] = cdx_final["cds_quoted_price"] - (cdx_final["accrued"] / 100000)
    return cdx_final

In [29]:
cdx_old = get_cdx_intrinsic_data(members=old_members)

In [30]:
cdx_old

security                security_des  \
0   CX855707 Curncy   ABUDHAB CDS USD SR 5Y D14   
1    CEGY1U5 Curncy     EGYPT CDS USD SR 5Y D14   
2   CT350188 Curncy    ARGENT CDS USD SR 5Y D14   
3    CBRZ1U5 Curncy    BRAZIL CDS USD SR 5Y D14   
4   CT965307 Curncy       KSA CDS USD SR 5Y D14   
5   CMLAY1U5 Curncy    MALAYS CDS USD SR 5Y D14   
6   CCHIN1U5 Curncy  CHINAGOV CDS USD SR 5Y D14   
7   CCHIL1U5 Curncy     CHILE CDS USD SR 5Y D14   
8    CCOL1U5 Curncy     COLOM CDS USD SR 5Y D14   
9    CINO1U5 Curncy     INDON CDS USD SR 5Y D14   
10   CPAN1U5 Curncy    PANAMA CDS USD SR 5Y D14   
11  CPERU1U5 Curncy      PERU CDS USD SR 5Y D14   
12  CSOAF1U5 Curncy    REPSOU CDS USD SR 5Y D14   
13  CPHIL1U5 Curncy    PHILIP CDS USD SR 5Y D14   
14  CTURK1U5 Curncy    TURKEY CDS USD SR 5Y D14   
15   CQTA1U5 Curncy     QATAR CDS USD SR 5Y D14   
16   CUKR1U5 Curncy    UKRAIN CDS USD SR 5Y D14   
17   CMEX1U5 Curncy       MEX CDS USD SR 5Y D14   

                                 issuer_bulk crncy  tenor  \
0   Emirate of Abu Dhabi United Arab Emirate   USD    5.0   
1                     Arab Republic of Egypt   USD    5.0   
2                     Argentine Republic/The   USD    5.0   
3              Federative Republic of Brazil   USD    5.0   
4                    Kingdom of Saudi Arabia   USD    5.0   
5                     Federation of Malaysia   USD    5.0   
6                 People's Republic of China   USD    5.0   
7                          Republic of Chile   USD    5.0   
8                       Republic of Colombia   USD    5.0   
9                      Republic of Indonesia   USD    5.0   
10                        Republic of Panama   USD    5.0   
11                          Republic of Peru   USD    5.0   
12                  Republic of South Africa   USD    5.0   
13               Republic of the Philippines   USD    5.0   
14                        Republic of Turkey   USD    5.0   
15                            State of Qatar   USD    5.0   
16                                   Ukraine   USD    5.0   
17                     United Mexican States   USD    5.0   

    upfront_format_indicator   px_last  upfront_last  sw_spread   wgt  \
0                          0    46.500 -2.257847e+00      100.0  0.02   
1                          1   978.452  2.900000e+01      100.0  0.02   
2                          1  5220.165  6.925000e+01      500.0  0.02   
3                          0   249.000  6.161054e+00      100.0  0.12   
4                          0    47.500 -2.213741e+00      100.0  0.02   
5                          0    78.500 -8.545615e-01      100.0  0.03   
6                          0    75.250 -1.005464e+00      100.0  0.12   
7                          0   136.000  1.604808e+00      100.0  0.03   
8                          0   260.000  6.626557e+00      100.0  0.06   
9                          0   111.000  5.484662e-01      100.0  0.09   
10                         0   138.000  1.688236e+00      100.0  0.02   
11                         0   126.000  1.186126e+00      100.0  0.03   
12                         0   281.000  7.394803e+00      100.0  0.12   
13                         0    98.500  1.421085e-14      100.0  0.02   
14                         0   753.000  2.312569e+01      100.0  0.12   
15                         0    47.500 -2.213741e+00      100.0  0.02   
16                         0   100.000  0.000000e+00      100.0  0.02   
17                         0   158.500  2.516883e+00      100.0  0.12   

    cds_quoted_price   accrued  cds_quoted_price_net  
0         102.324057  -24444.0            102.568497  
1          71.000000  -24444.0             71.244440  
2          30.750000 -122222.0             31.972220  
3          93.916865  -24444.0             94.161305  
4         102.279910  -24444.0            102.524350  
5         100.919184  -24444.0            101.163624  
6         101.059457  -24444.0            101.303897  
7          98.478722  -24444.0             98.723162 

In [31]:
cdx_old['wgt'] @ cdx_old['cds_quoted_price']

93.12470418945499

In [18]:
cdx_new = get_cdx_intrinsic_data(members=new_members)

In [19]:
cdx_new['wgt'] @ cdx_new['cds_quoted_price_net']

93.36740222689227

In [ ]:
if 'spread' in members and override_spread:
    spread_override = members.loc[members['spread'].isna(), :]
    cdx_data.loc[cdx_data['security'].isin([*spread_override['security']]), 'px_last'] = [*spread_override['spread']]

In [38]:
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)

In [54]:
cds_dg = DataGrid(
    cds_ref.query("tenor=='5Y' and currency=='USD'")[['security', 'name']].sort_values('name'), 
    column_widths={'security': 130, 'name': 250}, 
    selection_mode='cell'
)

In [55]:
cds_dg

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, column_widths={'security': 130, 'nam…

In [37]:
DataGrid(new_members, selection_mode='cell', editable=True)

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…